In [ ]:
%pip install torchvision 

In [9]:
import torch
from torch import nn, optim
from torch.nn import functional as F
import torch.autograd as autograd
torch.manual_seed(1)
START_TAG = "<START>"
STOP_TAG = "<STOP>"

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()

def generate_batch(df, size):
    sent_list = []
    embeds_list = []
    tags_list = []
    pos_list = []
    char_list = []
    for item in df:
        sent_list.append(item[0])
        embeds_list += item[1]
        tags_list += item[2]
        pos_list += item[3]
        char_list += item[4]
        if len(sent_list) == size:
            yield sent_list, embeds_list, tags_list, pos_list, char_list
            sent_list = []
            embeds_list = []
            tags_list = []
            pos_list = []
            char_list = []

# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

CHAR_EMBEDDING_DIM = 100
CHAR_HIDDEN_DIM = 10
CHAR_TARGET_SIZE = 50
EMBEDDING_DIM = 168
HIDDEN_DIM = 10
tag_to_ix = {'<START>': 0, '<STOP>': 1, 'O': 2, 'B-ORGANIZATION': 3, 'I-ORGANIZATION': 4, 'B-SYMPTOM_AND_DISEASE': 5, 'I-SYMPTOM_AND_DISEASE': 6, 'B-LOCATION': 7, 'B-DATE': 8, 'B-PATIENT_ID': 9, 'B-AGE': 10, 'B-NAME': 11, 'I-DATE': 12, 'B-JOB': 13, 'I-LOCATION': 14, 'B-TRANSPORTATION': 15, 'B-GENDER': 16, 'I-TRANSPORTATION': 17, 'I-JOB': 18, 'I-NAME': 19, 'I-AGE': 20, 'I-PATIENT_ID': 21}

class CharEmbedding(nn.Module):
    def __init__(self, char_dim, char_hidden_dim, char_size, char_target_size):
        super(CharEmbedding, self).__init__()
        self.char_embeddings = nn.Embedding(char_size, char_dim)
        self.BiLSTM = nn.LSTM(char_dim, char_hidden_dim // 2, num_layers=1, batch_first=True, bidirectional=True)
        self.hidden2out = nn.Linear(char_hidden_dim, char_target_size)

    def forward(self, word):
        embeds = self.char_embeddings(word)
        lstm_out, _ = self.BiLSTM(embeds)
        output = torch.mean(self.hidden2out(lstm_out), dim = 0)
        return output


class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        #self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        hidden = (torch.randn(2, 1, self.hidden_dim // 2),torch.randn(2, 1, self.hidden_dim // 2))
        return hidden

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas
        forward_var = forward_var
        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = sentence.view(len(sentence), 1, -1)     #self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq


char_model = CharEmbedding(CHAR_EMBEDDING_DIM, CHAR_HIDDEN_DIM, 176, CHAR_TARGET_SIZE)
model = BiLSTM_CRF(7886, tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)



optimizer = optim.Adam([
                {'params': char_model.parameters()},
                {'params': model.parameters(), 'lr': 1e-3}
            ], lr=1e-2)




model.load_state_dict(torch.load('file.pt'))

<All keys matched successfully>

In [5]:
import pickle
import numpy as np

with open('test.txt', 'rb') as pickle_file:
    test = pickle.load(pickle_file)

test = np.array(test, dtype = 'object')

In [ ]:
test[0]

In [20]:
f = open("predict.txt", "w", encoding="utf-8")

for index, (sent, embeds, tags, pos, char) in enumerate(test):
    embeds = torch.tensor(embeds, dtype=torch.float32)
    pos = torch.tensor(pos, dtype=torch.float32)
    char_embed = torch.empty(size = (len(char), 50))
    for i, word in enumerate(char):
        word = torch.tensor(word, dtype = torch.long)
        output = char_model(word)
        char_embed[i] = output
    valid_input = torch.cat((embeds,char_embed, pos), 1)
    valid_targets = torch.tensor(tags)

    _, pred_tag = model(valid_input)
    pred_tag = [list(tag_to_ix.keys())[list(tag_to_ix.values()).index(tag)] for tag in pred_tag]
    true_tag = [list(tag_to_ix.keys())[list(tag_to_ix.values()).index(tag)] for tag in valid_targets]
    for word, tags, pred in zip(test[index][0], true_tag, pred_tag):
        f.write("{} a {} {}\n".format(word, tags, pred))

In [ ]:
%pip install conlleval
from conlleval import evaluate

In [22]:
!python conlleval.py < predict.txt > result.txt